In [63]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import json

In [64]:
url ='https://dublin.ie/whats-on/'

In [65]:
#Opening connection , grabbing the page
uClient = uReq(url)
page_html = uClient.read()

In [66]:
uClient.close()

In [67]:
#Parsing
page_soup = soup(page_html, "html.parser")
#print(page_soup)

In [68]:
# Finding each events
article = page_soup.findAll('article',{"class":"event card "})
#print(article)

In [69]:
import time
start_time = time.time()

data_list = []

for a in article:
    div = a.findAll('div',{"class":"text"})
#     print(div)
    v = div[0].a["href"]
    url2 = v
    #Opening connection to second page, grabbing the page
    uClient = uReq(url2)
    page_html = uClient.read()
    uClient.close()
    #Parsing2
    page_soup = soup(page_html, "html.parser")
#     print(page_soup)
    div = page_soup.findAll('div',{"class":"event-info"})
    data = {}
    data["title"] = page_soup.h1.text
    data["desc"] = page_soup.findAll('div',{"class":"shortened"})[0].text.strip()
    data["read_more"] = url2
    data["category"] = a["data-categories"]
    details = div[0].findAll('dd')
    a = div[0].findAll('dt')
    for i in range(len(a)):
        data[a[i].text] = details[i].text
    data_list.append(data)

#print(data_list)
print("--- %s seconds ---" % (time.time() - start_time))

--- 44.26045370101929 seconds ---


In [70]:
len(data_list)

110

In [71]:
import pandas as pd
df = pd.DataFrame(data_list)

In [72]:
df.describe()

,Address:,Date:,Price:,Time:,category,desc,read_more,title
count,109,110,107,110,110,110,110,110
unique,64,81,66,58,39,110,110,110
top,"RDS Venue - Main Hall Complex (Halls 1-4 & 6),...",Thursday 17th October,Free,Varies,music-and-comedy,Other People's Practices Juncture One – artist...,https://dublin.ie/whats-on/listings/ncad-open-...,Tommy Tiernan
freq,10,4,19,24,19,1,1,1


In [73]:
#df

In [74]:
df.isnull().sum()

Address:     1
Date:        0
Price:       3
Time:        0
category     0
desc         0
read_more    0
title        0
dtype: int64

In [75]:
dups = df.duplicated()
df[dups]

,Address:,Date:,Price:,Time:,category,desc,read_more,title


In [76]:
date_split = df["Date:"].str.split('-',expand=True)
df["start_date"] = date_split[0]
df["end_date"] = date_split[1]
#df

In [77]:
# e = df["Start_date"][0]
# f = "Monday 30 September"
# from datetime import datetime
# format = "%A %d %B"
# print(datetime.strptime(f, format))

In [109]:
new_df = pd.concat([df, df['category'].str.get_dummies(sep=',')], axis=1)
#new_df

In [110]:
main_cols = ["Address:","Date:","Price:","Time:","desc","read_more","title","start_date","end_date"]
categories = df['category'].str.get_dummies(sep=',').columns.tolist()
new_df = new_df.drop('category',axis=1)
#new_df

In [107]:
melt_df = pd.melt(new_df, id_vars=main_cols, value_vars=categories)
#melt_df

In [88]:
melt_df = melt_df[melt_df["value"] == 1]

In [89]:
melt_df = melt_df.drop(['value','Date:'],axis=1)

In [105]:
melt_df = melt_df.rename(columns={"Address:":"address","Price:":"price","Time:":"time","variable":"category"})
melt_df = melt_df.reset_index(drop=True)
#melt_df

In [106]:
out = melt_df.to_json(orient='index')
#out